In [2]:
import cobra

In [3]:
FBA_model = cobra.io.read_sbml_model('H_volcanii_biomass_subrxns.sbml')

In [4]:
biomass_rxn = FBA_model.reactions.get_by_id('biomass0')
DNA = FBA_model.metabolites.get_by_id('DNA_c0')
RNA = FBA_model.metabolites.get_by_id('RNA_c0')
cofactors = FBA_model.metabolites.get_by_id('cofactors_c0')
lipid = FBA_model.metabolites.get_by_id('Lipid_c0')
energy = FBA_model.metabolites.get_by_id('Energy_c0')
salt = FBA_model.metabolites.get_by_id('Salt_c0')
glycosylation = FBA_model.metabolites.get_by_id('Glycosylation_c0')
protein = FBA_model.metabolites.get_by_id('Protein_c0')
biomass = FBA_model.metabolites.get_by_id('cpd11416_c0')

In [5]:
print(biomass_rxn)

biomass0: 35.5403092430435 cpd00001_c0 + 40.1101757365074 cpd00002_c0 + 0.00309646685192537 cpd00003_c0 + 0.00309646685192537 cpd00006_c0 + 0.00309646685192537 cpd00010_c0 + 0.00309646685192537 cpd00015_c0 + 0.00309646685192537 cpd00016_c0 + 0.00309646685192537 cpd00017_c0 + 0.219088153012743 cpd00023_c0 + 0.00309646685192537 cpd00028_c0 + 0.00309646685192537 cpd00030_c0 + 0.509869786991038 cpd00033_c0 + 0.00309646685192537 cpd00034_c0 + 0.427934380173264 cpd00035_c0 + 0.135406821203723 cpd00038_c0 + 0.285438020490179 cpd00039_c0 + 0.200830806928348 cpd00041_c0 + 0.00309646685192537 cpd00042_c0 + 0.00309646685192537 cpd00048_c0 + 0.246696822701341 cpd00051_c0 + 0.0841036156544863 cpd00052_c0 + 0.219088153012743 cpd00053_c0 + 0.179456352975885 cpd00054_c0 + 0.00309646685192537 cpd00056_c0 + 0.00309646685192537 cpd00058_c0 + 0.127801422590767 cpd00060_c0 + 0.0908319049068452 cpd00062_c0 + 0.00309646685192537 cpd00063_c0 + 0.0472019191450218 cpd00065_c0 + 0.154519490031345 cpd00066_c0 + 0

In [6]:
biomass_rxn.subtract_metabolites(biomass_rxn.metabolites)

In [7]:
print(biomass_rxn)

biomass0:  --> 


In [8]:
react_dict = {}
react_dict[DNA] = -1
react_dict[RNA] = -1
react_dict[cofactors] = -1
react_dict[lipid] = -1
react_dict[energy] = -1
react_dict[salt] = -1
react_dict[glycosylation] = -1
react_dict[protein] = -1
react_dict[biomass] = 1
biomass_rxn.add_metabolites(react_dict)
print(biomass_rxn)

biomass0: DNA_c0 + Energy_c0 + Glycosylation_c0 + Lipid_c0 + Protein_c0 + RNA_c0 + Salt_c0 + cofactors_c0 --> cpd11416_c0


In [9]:
##Build the medium (this time with thiamin and biotin added
## --but no need for K+, Mg2+, Ca2+, Zn2+ since they're not in the new biomass)
## and make sure to use Fe3+ instead of Fe2+ since the cell only has a transporter for the former
##glucose, ammonia, O2, PO4, sulfate, Fe3+, Co2+, H+, H2O, thiamin, biotin
med_components = ['cpd00190_e0','cpd00013_e0','cpd00007_e0','cpd00009_e0','cpd00048_e0','cpd10516_e0','cpd00149_e0','cpd00067_e0','cpd00001_e0','cpd00305_e0','cpd00104_e0']
med = {}

for component in med_components:
	try:
		met = FBA_model.metabolites.get_by_id(component)
	except:
		print('model to be filled has no ' + component + ': Adding it!')
		met = cobra.Metabolite(component)
	try:
		exchange = FBA_model.reactions.get_by_id('EX_' + component)
	except:
		exchange = FBA_model.add_boundary(met)
		print('new exchange reaction added: EX_' + component)
	exch_id = exchange.id
	med[exch_id] = 300
		
FBA_model.medium = med




new exchange reaction added: EX_cpd00190_e0
new exchange reaction added: EX_cpd00048_e0


In [10]:
##Does it grow now??
FBA_model.solver = 'glpk'
solution = FBA_model.optimize()
print(solution.f)

0.0


In [9]:
## It doesn't grow, but I found that there is no ATP synthase 
## (hence no way to interconvert ADP/ATP with redox and ion equilibrium across the membrane)
rxn_atp_syn = cobra.Reaction('rxn_atp_syn', name='ATP synthase')
atp_syn_dict = {}
H_extra = FBA_model.metabolites.get_by_id('cpd00067_e0')
H_intra = FBA_model.metabolites.get_by_id('cpd00067_c0')
ADP = FBA_model.metabolites.get_by_id('cpd00008_c0')
phosphate = FBA_model.metabolites.get_by_id('cpd00009_c0')
ATP = FBA_model.metabolites.get_by_id('cpd00002_c0')
water = FBA_model.metabolites.get_by_id('cpd00001_c0')
atp_syn_dict[H_extra] = -4
atp_syn_dict[H_intra] = 3
atp_syn_dict[ADP] = -1
atp_syn_dict[phosphate] = -1
atp_syn_dict[ATP] = 1
atp_syn_dict[water] = 1
rxn_atp_syn.add_metabolites(atp_syn_dict)
FBA_model.add_reaction(rxn_atp_syn)



In [12]:
##try growth again
FBA_model.solver = 'glpk'
solution = FBA_model.optimize()
print(solution.f)

0.0


In [10]:
print(rxn_atp_syn)

rxn_atp_syn: cpd00008_c0 + cpd00009_c0 + 4 cpd00067_e0 --> cpd00001_c0 + cpd00002_c0 + 3 cpd00067_c0


In [11]:
rxn_atp_syn.lower_bound = -1*rxn_atp_syn.upper_bound

In [15]:
FBA_model.solver = 'glpk'
solution = FBA_model.optimize()
print(solution.f)

0.0


In [29]:
##still doesn't work, find out which subreaction is the problem
def try_as_biomass(met):
    react_dict = {}
    test_biomass = FBA_model.reactions.get_by_id('test_biomass')
    biomass = FBA_model.metabolites.get_by_id('cpd11416_c0')
    react_dict[biomass] = 1
    react_dict[met] = -1
    old_mets = test_biomass.metabolites
    test_biomass.subtract_metabolites(old_mets)
    test_biomass.add_metabolites(react_dict)
    FBA_model.objective = {FBA_model.reactions.test_biomass: 1}

    solution = FBA_model.optimize()
    if solution.f > 0.0001:
        return 'Success!'
    else:
        return 'Failed :-('

test_biomass = cobra.Reaction('test_biomass')
FBA_model.add_reaction(test_biomass)

for met in biomass_rxn.metabolites:
    if not met.id == 'cpd11416_c0':
        print(str(met.id) + ': ' + try_as_biomass(met))


Ignoring reaction 'test_biomass' since it already exists.


DNA_c0: Failed :-(
RNA_c0: Success!
cofactors_c0: Failed :-(
Lipid_c0: Success!
Energy_c0: Success!
Salt_c0: Failed :-(
Glycosylation_c0: Failed :-(
Protein_c0: Failed :-(


In [22]:
print(biomass_rxn.metabolites)

{<Metabolite DNA_c0 at 0xdafb828>: -1, <Metabolite RNA_c0 at 0xdafb940>: -1, <Metabolite cofactors_c0 at 0xdafb9e8>: -1, <Metabolite Lipid_c0 at 0xdafba20>: -1, <Metabolite Energy_c0 at 0xdafbb70>: -1, <Metabolite Salt_c0 at 0xdafbcc0>: -1, <Metabolite Glycosylation_c0 at 0xdafbd68>: -1, <Metabolite Protein_c0 at 0xdafbda0>: -1, <Metabolite cpd11416_c0 at 0xdad1ac8>: 1}


In [32]:
protein_syn = FBA_model.reactions.get_by_id('rxn_protein')
for amino_acid in protein_syn.metabolites:
    if not amino_acid.id == 'Protein_c0':
        print(str(amino_acid.id) + ': ' + try_as_biomass(amino_acid))

cpd00023_c0: Success!
cpd00033_c0: Success!
cpd00035_c0: Success!
cpd00039_c0: Failed :-(
cpd00041_c0: Success!
cpd00051_c0: Success!
cpd00053_c0: Success!
cpd00054_c0: Success!
cpd00060_c0: Success!
cpd00065_c0: Success!
cpd00066_c0: Success!
cpd00069_c0: Success!
cpd00084_c0: Success!
cpd00107_c0: Success!
cpd00119_c0: Success!
cpd00129_c0: Success!
cpd00132_c0: Success!
cpd00156_c0: Success!
cpd00161_c0: Success!
cpd00322_c0: Success!


In [35]:
cofactor_syn = FBA_model.reactions.get_by_id('rxn_cofactors')
for cofactor in cofactor_syn.metabolites:
    if not cofactor.id == 'cofactors_c0':
        print(str(cofactor.id) + ': ' + try_as_biomass(cofactor))

cpd00002_c0: Success!
cpd00003_c0: Success!
cpd00006_c0: Success!
cpd00010_c0: Success!
cpd00015_c0: Success!
cpd00022_c0: Success!
cpd00023_c0: Success!
cpd00087_c0: Failed :-(
cpd00166_c0: Success!
cpd00201_c0: Failed :-(
cpd00305_c0: Success!
cpd00347_c0: Failed :-(
cpd00557_c0: Success!
cpd11606_c0: Failed :-(


In [38]:
DNA_syn = FBA_model.reactions.get_by_id('rxn_DNA')
for cofactor,coeff in DNA_syn.metabolites.items():
    if coeff < 0:
        print(str(cofactor.id) + ': ' + try_as_biomass(cofactor))

cpd00115_c0: Success!
cpd00241_c0: Failed :-(
cpd00356_c0: Success!
cpd00357_c0: Success!


In [39]:
glycosylation = FBA_model.reactions.get_by_id('rxn_glycosylation')
for substrate,coeff in glycosylation.metabolites.items():
    if coeff < 0:
        print(str(substrate.id) + ': ' + try_as_biomass(substrate))

cpd00017_c0: Success!
cpd00026_c0: Failed :-(
cpd00083_c0: Success!
cpd00144_c0: Failed :-(
cpd00472_c0: Failed :-(


In [12]:
cobra.io.write_sbml_model(FBA_model,'H_volcanii_biomass_replaced.sbml')